In [7]:
#200201020 begüm erva şahin-- 200201050 seda Nur ekici
#python kod
import requests
from bs4 import BeautifulSoup
import pandas as pd
from csv import writer
import pymongo
from pymongo import MongoClient

#ilk üç satır mongoya dinamik bağlanmak içindir
client = MongoClient('mongodb://localhost:27017')
db = client.YAZLAB1
collection = db.products

header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 OPR/91.0.4516.65"
}


#TRENDYOL webscharping 
# Mongo db ile dinamaik bağlantı oluşturuldu. .Mongodaki veriler json formatına aktarılarak web sitesine eklendi
listem=[]#en son tün özellikler listede toplandı
url = "https://www.trendyol.com/sr?q=laptop&qt=laptop&st=laptop&os=1"#sitenin linkini alıyor ve url ye atıyor
response = requests.get(url)
content = response.content
soup1 = BeautifulSoup(content, "html.parser")
for a in range(1,2):
    
    urunler = soup1.find_all("div", {"class": "prdct-cntnr-wrppr"})
    for urun in urunler:
        urun_link1 = urun.find_all("div", attrs={"class": "with-campaign-view"})
        for i in urun_link1:
            link = i.find("div", {"class": "p-card-chldrn-cntnr card-border"})#linkin adresine gider ve linki alır
            link = "https://www.trendyol.com" + link.a.get("href")#linkin çalışması için linkin başını tamamlar
            detail = requests.get(link, headers=header)
            soup2 = BeautifulSoup(detail.content, "html.parser")#linkten adrese gider

            # print(link)
            price = soup2.find("span", {"class": "prc-dsc"}).text#fiyat bilgisini alır
            name_ = soup2.find("h1", {"class": "pr-new-br"})
            categoryId= name_.a.text
            name = name_.find("span").text#adını alır
            image=soup2.find("div",{"class":"flex-container"}).img.get("src")#resmini alır
            description='Trendyol'#site adını alır

            teknikayrinti = soup2.find_all("ul", {"class": "detail-attr-container"})
 #ikinci for ürüne tıklayınca yeni açılan linkteki detaylardan aldığım için yapıldı
            for i in teknikayrinti:
                try:
                   
                    islemci = i.find_all("b")[0].get_text().strip()#işlemciyi alır
                    islemci_nesli = i.find_all("b")[26].get_text().strip()#islemci neslini alır
                    Ram = i.find_all("b")[4].get_text().strip()#ram i alır
                    disk_boyutu = i.find_all("b")[16].get_text().strip()#disk boyutunu alır
                    ekran_boyut = i.find_all("b")[20].get_text().strip()#ekran boyutunu alır
                    
                except:
                    continue

                listem.append([name,price,islemci, Ram,categoryId,ekran_boyut,link,image,description])
                #aldığımız özellikleri listem adındaki bir listeye aktarır
#buradaki for mongoya tüm verileri aktarmak içindir 
# listenin içinde dolaşarak sırayla verileri alır
for i in listem:
    bson = {

    'name':i[0],
    'price':i[1],
    'islemci':i[2],
    'Ram':i[3],
    'categoryId':i[4],
    'ekran_boyut':i[5],
    'link':i[6],
    'image':i[7],
    'description':i[8]
   }
    response=collection.insert_one(bson)#mongoDB ye aktarır
    
    
    
    
    #*****************************************************************************************************************************            
    
#vatan webscharping
#ilk üç satır mongoya dinamik bağlanmak içindir 

client = MongoClient('mongodb://localhost:27017')
db = client.YAZLAB1
collection = db.products
listem=[]
for sayfa in range(1,2):#siteden 20 sayfa alır
      
   url = "https://www.vatanbilgisayar.com/notebook/?page={}".format(sayfa)#sitenin linkini alır

   parser=BeautifulSoup(requests.get(url).content,"html.parser")

   veri=parser.find("div",{"class":"wrapper-product wrapper-product--list-page clearfix"})\
   .find_all("div",{"class":"product-list product-list--list-page"})#verilen div lerinin içindeki classlara gider


   for i in veri:
      urunbilgisi=i.find("div",{"class":"product-list__product-name"}).find("h3").text
      yeniliste=urunbilgisi.split("-")#(-) ile ürün ismini parçalar
      name=yeniliste[0]#parcalanan ifadenin ilk kelimesini alır
      yeniliste2=isim.split(" ")#ilk ifadeyi bosluklardan parcalar
      marka=yeniliste2[0]#parcaladıgı ifadedeki ilk kelimeyi alır ve bu marka adıdır
     # print(isim2)
      
      ram=yeniliste[1]#ram bilgisini alır
      #print(ram)

      islemci=yeniliste[2]#işlemci bilgisini alır
      ekranBoyutu=yeniliste[3]#ekran boyutu bilgisini alır
      isletimsistemi=yeniliste[4]#işletim sistemi bilgisini alır
  
      price=i.find("span",{"class":"product-list__price"}).text#ürünün fiyat bilgisini alır
      modelNoliste=i.find("div",{"class":"product-list__product-code"}).text.split()
      modelNo=modelNoliste[0]#model no sunu alır
      print(modelNo)
      
      header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"}
  #sitede ayrı linkler olduğu için link birleştirme yapıldı 
      for detaylar in veri:
         linksonu = detaylar.a.get("href")#siteddeki linki alır
         linkbası = "https://www.vatanbilgisayar.com"#linkin basına ekler
         linkk =linkbası+linksonu#verilen ifadeleri ekler
         detay=requests.get(linkk,headers = header)
        
         detay_soup=BeautifulSoup(detay._content,"lxml")
         teknik_ayrıntılar= detay_soup.find_all("ul",{"class":"pdetail-property-list"})#teknik ayrıntıları alır
         
      puan=i.find("div",{"class":"product-list__content"}).find("div",{"class":"wrapper-star"}).text.strip()#puan bilgisini alır
      
      description="VATAN"#site ismini yazar
      
      
      try:
       image=i.img.get("data-src")#resim bilgisini alır
      except (IndexError,TypeError, AttributeError):
       pass    
      #print(foto)

      listem.append([name,ram,puan,marka,linkk,price,image,islemci,ekranBoyutu,description,modelNo])
      #oluşturduğumuz listeye siteden aldığımız bilgileri eklerriz
for i in listem:
              bson = {
            
                  'name':i[0],#listenin ilk elemanı
                  'ram':i[1],#listenin ikinci elemanı
                  'puan':i[2],#listenin üçüncü elemanı
                  'categoryId':i[3],#listenin dörduncu elemanı
                  'linkk':i[4],#listenin beşinci elemanı
                  'price':i[5],#listenin altıncı elemanı
                  'image':i[6],#listenin yedinci elemanı
                  'islemci':i[7],#listenin sekizinci elemanı
                  'ekranBoyutu':i[8],#listenin dokuzuncu elemanı
                  'description':i[9],#listenin onuncu elemanı
                  'modelNo':i[10]}#listenin onbirinci elemanı
              parser=collection.insert_one(bson)
             #mongoDB ye aktarır
    
    

    
 #*****************************************************************************************************************************     
    
    
        
#HEPSİBURADA webscharping 
#ilk üç satır mongoya dinamik bağlanmak içindir 

client = MongoClient('mongodb://localhost:27017')
db = client.YAZLAB1
collection = db.products

listisim = []
listmarka = []
listoriginal_fiyat = []
listpuan = []
listdegerlendirme = []
listresim = []
listekranhafiza = []
listboyut = []
listekrankarti = []
listisletimsistemi = []
listislemci = []
listram = []
listharddiskkapasitesi = []
listekrankarti = []
listisletimsistemi = []
listislemcitipi= []
listram = []
listislemcitipi = []
listlink = []
listgarantitipi=[]
listsiteismi=[]
#yukarıda farklı bir yoldan giderek önce ayrı ayrı listelendi sonra tek bir zip halinde listendi
a=2
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
r = requests.get("https://www.hepsiburada.com/laptop-notebook-dizustu-bilgisayarlar-c-98?sayfa=" + str(a) + "",
                     headers=headers)

while a<=3:
    soup = BeautifulSoup(r.content, "lxml")
    st1 = soup.find("div", attrs={"class": "MORIA-voltran-body voltran-body ProductList"})#verilen classa gider

    st3 = st1.find_all("li", attrs={"class": "productListContent-zAP0Y5msy8OHn5z7T_K_"})#verilen classa gider
    for detaylar in st3:
        try:
            link_sonu = detaylar.a.get("href")#sitedeki adresten linki alır
            link_bası = "https://www.hepsiburada.com"#linki tanımlamak için başına eklememiz gerek
            link = link_bası + link_sonu#yukarıdaki iki ifadeyi birleştirir
            listlink.append(link)
            siteismi="HEPSİBURADA"#site ismini alır
            listsiteismi.append(siteismi)
  #yukarıda linli parçalayıp birleştirdim çünkü site linki ayrı ayrı vermiş      
        except:
            link="Ürün Bulunamadı"
            listlink.append(link)#linki listeye aktarır
            #link için sorgu yapar
            
        r1 = requests.get(link, headers=headers)
        soup1 = BeautifulSoup(r1.content, "lxml")
        try:
            isim = soup1.find("span", attrs={"class": "product-name"}).text#ürün ismini alır
            listisim.append(isim)#listeye aktarır
        except:
            isim ="Ürün Bulunamadı"
            listisim.append(isim)
            #isim için sorgu yapar
        try:
            marka = soup1.find("a", attrs={
                "data-bind": "attr: {'data-hbus': userInformation() && userInformation().userId && isEventReady()? productDetailHbus('BrandClick') : '' }"}).text
                #markayı alır
            listmarka.append(marka)#markayı listeye ekler
        except: 
             marka="Ürün Bulunamadı"  
             listmarka.append(marka)#markayı listeye ekler
        try:
            original_fiyat = soup1.find("del", attrs={"id": "originalPrice"}).text#fiyati alır
            listoriginal_fiyat.append(original_fiyat)
        except:
            original_fiyat="Ürün Bulunamadı"
            listoriginal_fiyat.append(original_fiyat)
        try:
            puan = soup1.find("span", attrs={"class": "rating-star"}).text.strip()#puanı alır
            listpuan.append(puan)#puanı listeye ekler
        except:
            puan = "Ürün Bulunamadı"
            listpuan.append(puan)
        try:

            degerlendirme_sayisi = soup1.find("div", attrs={"id": "comments-container"}).text.strip().strip()
            #değerlendirmeleri alır
            listdegerlendirme.append(degerlendirme_sayisi)#değerlendirmeleri listeye aktarır
        except:
            degerlendirme_sayisi = "Ürün Bulunamadı"
            listdegerlendirme.append(degerlendirme_sayisi)#değerlendirmeleri listeye aktarır
        try:
            urun_resmi = soup1.img.get("src")#ürün resmini alır
            listresim.append(urun_resmi)#listeye ürün resmini ekler
        except:
            urun_resmi="Ürün Bulunamadı"
            listresim.append(urun_resmi)#listeye ürün resmini ekler
        # urun ozellikleri cekme
        teknikayrinti = soup1.find("div", attrs={"data-bind": "css: {'hidden': keyFeatures().length}"})
        liste = teknikayrinti.find("ul", attrs={"class": ""})
        eklenecek = liste.find_all("span")
        listem = []
        for i in eklenecek:
            yenieleman = i.text.strip()
            listem.append(yenieleman)
        

        for j in listem:
            try:
                garantitipi=listem[0]#listenin sıfırıncı elemanı
                listgarantitipi.append(garantitipi)
            except:
                garantitipi="Ürün Bulunamadı"
                listgarantitipi.append(garantitipi)
            try:
                harddiskkapasitesi = listem[1]#listenin birinci elemanı
                listharddiskkapasitesi.append(harddiskkapasitesi)
            except:
                harddiskkapasitesi="Ürün Bulunamadı"
                listharddiskkapasitesi.append(harddiskkapasitesi)
            try:
                islemcitipi = listem[2]#listenin ikinci elemanı
                listislemcitipi.append(islemcitipi)
            except:
                islemcitipi="Ürün Bulunamadı"  
                listislemcitipi.append(islemcitipi)
            try:
                islemci= listem[3]#listenin üçüncü elemanı
                listislemci.append(islemci)
            except:
                islemci="Ürün Bulunamadı"
                listislemci.append(islemci)
            try:
                isletimsistemi = listem[4]#listenin dördüncü elemanı
                listisletimsistemi.append(isletimsistemi)
            except:
                isletimsistemi="Ürün Bulunamadı"
                listisletimsistemi.append(isletimsistemi)
            try:
                ram = listem[5]#listenin beşinci elemanı
                listram.append(ram)
            except:
                ram="Ürün Bulunamadı"
                listram.append(ram)
            try:
                ekrankartihafizasi = listem[6]#listenin altıncı elemanı
                listekranhafiza.append(ekrankartihafizasi)
            except:
                ekrankartihafizasi="Ürün Bulunamadı"
                listekranhafiza.append(ekrankartihafizasi)
            try:  
                ekranboyutu = listem[7]#listenin yedinci elemanı
                listboyut.append(ekranboyutu)
            except:
                ekranboyutu="Ürün Bulunamadı"
                listboyut.append(ekranboyutu)
            try:
                ekrankarti = listem[8]#listeninsekizinci elemanı
                listekrankarti.append(ekrankarti)
            except:
                ekrankarti="Ürün Bulunamadı"
                listekrankarti.append(ekrankarti)
            
    a = a + 1 
    #aşyağıda listeleri zip ile birleştirdim
yeniliste= list(
    zip(listisim, listmarka, listoriginal_fiyat,listlink, listpuan, listdegerlendirme,listresim,listgarantitipi,listharddiskkapasitesi, listislemcitipi,listislemci,listisletimsistemi,listram,listekranhafiza,listboyut,listekrankarti,listsiteismi))
for i in yeniliste:
    bson = {
        'name':i[0],
        'categoryId':i[1],
        'price':i[2],
        'link':i[3],
        'puan':i[4],
        'degerlendirme':i[5],
        'image':i[6],
        'garantitipi':i[7],
        'harddiskkapasitesi':i[8],
        'islemcitipi':i[9],
        'islemci':i[10],
        'isletim_sistemi':i[11],
        'ram':i[12],
        'ekranhafiza':i[13],
        'ekran_boyut':i[14],
        'ekrankarti':i[15],
        'description':i[16]
        
    }
    r=collection.insert_one(bson)  
    #mongoBD ye aktarır
    
    
#*****************************************************************************************************************************    
    
#N11 webscharping
#ilk üç satır mongoya dinamik bağlanmak içindir 

client = MongoClient('mongodb://localhost:27017')
db = client.YAZLAB1
collection = db.products

listem=[]#liste tanımladık
header= {
   "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 OPR/91.0.4516.65"
}

for a in range(1,2):
    url = "https://www.n11.com/bilgisayar/dizustu-bilgisayar?pg="+str(a)#linki alır

    html = requests.get(url).content
    soup = BeautifulSoup(html, "html.parser")

    list = soup.find_all("li",{"class":"column"})
   
    for li in list:
    #marka model model no işletimsistemi  iştipi iş nesli ram disk boyutu diktütü ekran  site ismi
        name = li.div.a.h3.text.strip()#adını alır
        link = li.div.a.get("href")#linki alır
        description="N11"#site ismini alır
        # newprice = li.find("div",{"class":"priceContainer"}).div.find_all("span")[1].text.strip()
        price = li.find("div",{"class":"priceContainer"}).div.find_all("span")[0].text.strip()#fiyat bilgisini alır
        
        puan= li.find("div", {"class":"proDetail"}).find_all("span")[1].text.strip()#puan bilgisini alır
   # print(isim+puan+link+newprice+puan)

#bu kodda tek link üstünden ikinci for aratıldı.
        detay=requests.get(link,headers=header).content
        soup2=BeautifulSoup(detay,"html.parser")

        list2=soup2.find_all("div",{"class":"unf-p-lBox"})
        a=0
        for div in list2:
            try:
                islemci=div.find("div",{"class":"unf-attribute-cover"}).find_all("strong")[0].get_text().strip()#iişlemciyi alır
            except (IndexError,TypeError, AttributeError):
                
                pass
            try:
                bellek=div.find("div",{"class":"unf-attribute-cover"}).find_all("strong")[1].get_text().strip()#bellek bilgisini alır
            except (IndexError,TypeError, AttributeError):
                bellek= ""
                pass
            try:
                bellek=div.find("div",{"class":"unf-attribute-cover"}).find_all("strong")[1].get_text().strip()#adını alır
                #print(name)
                a=a+1
            except (IndexError,TypeError, AttributeError):
                pass
            try:
                categoryId=div.find("div",{"class":"unf-attribute-cover"}).find_all("strong")[2].get_text().strip()#category ıd alır
            except (IndexError,TypeError, AttributeError):
                pass
            try:
                boyut=div.find("div",{"class":"unf-attribute-cover"}).find_all("strong")[3].get_text().strip()#boyut bilgisini alır
            except (IndexError,TypeError, AttributeError):
                pass
            try:
                isletim_sistemi=div.find("div",{"class":"unf-attribute-cover"}).find_all("strong")[4].get_text().strip()#işletim sistemi bilgisini alır
            except (IndexError,TypeError, AttributeError):
                pass
            try:
                image=div.img.get("data-src")#ürün resmini alır
            except (IndexError,TypeError, AttributeError):
                pass    
                
            # print(islemci+bellek+marka+boyut+isletim_sistemi+foto)
            listem.append([name,puan,price,islemci,bellek,categoryId,boyut,isletim_sistemi,link,image,description])
            #oluşturduğumuz listeye üründen aldıpımız bilgileri ekledik
            for i in listem:
                bson = {
                    'name':i[0],
                    'puan':i[1],
                    'price':i[2],
                    'islemci':i[3],
                    'bellek':i[4],
                    'categoryId':i[5],
                    'boyut':i[6],
                    'isletim_sistemi':i[7],
                    'link':i[8],
                    'image':i[9],
                    'description':i[10]}
                
                    
            html=collection.insert_one(bson)
            #mongoDB ye aktarır
                

            
            
            
            

            

A315-23-R0U2
81VU00FQTX
82H80252TX
82BA0028TK
MBD15/I511/8/256
X515EA-BR2026
X515EA-BQ1830W
82H80250TX
X415MA-EK656W
X515EA-BQ511W
A715-42G-R016
82H802RKTX
53013DBB
TMP215-52G-52WC
31.8049
MLY33TU/A
